In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import os, yaml
from easydict import EasyDict
import pickle
import hashlib
from datetime import datetime

import torch
from torch.utils.data import DataLoader

from sklearn.model_selection import KFold

from dataloader.bci_compet import get_dataset
from dataloader.bci_compet import BCICompet2aIV

from model.litmodel import LitModel
from model.attn_conditioned import ATTNConditioned
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_lightning import Trainer, seed_everything


from utils.setup_utils import (
    get_device,
    get_log_name,
)
from utils.training_utils import get_callbacks

torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

/home/devuser/.local/lib/python3.8/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /home/devuser/.local/lib/python3.8/site-packages/braindecode/models/eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_outputs' is missing.
  self._warn(
/home/devuser/.local/lib/python3.8/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /home/devuser/.local/lib/python3.8/site-packages/braindecode/models/eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_chans' is missing.
  self._warn(
/home/devuser/.local/lib/python3.8/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /home/devuser/.local/lib/python3.8/site-packages/braindecode/models/eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstrin

In [3]:
CACHE_ROOT = 'cache'

config_name = 'bcicompet2a_config'
return_subject_id = True

with open(f'configs/{config_name}.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
    args = EasyDict(config)

def hash(config):
    return 'base'
    config_string = str(sorted(config.items())).encode()
    return hashlib.sha256(config_string).hexdigest()[:8]
config_hash = hash(config)

In [4]:
def load_dataset(args, return_subject_id=False):
    datasets = {}
    for subject_id in range(0,9):
        args['target_subject'] = subject_id
        datasets[subject_id] = BCICompet2aIV(args, return_subject_id=return_subject_id)
    return datasets

hash_path = os.path.join(CACHE_ROOT, f'{config_name}_{config_hash}_{str(int(return_subject_id))}.pkl')

if not os.path.isfile(hash_path):
    print('Cache miss, generating cache')
    datasets = load_dataset(args, return_subject_id=return_subject_id)
    with open(hash_path, 'wb') as file:
        pickle.dump(datasets, file)
else:
    print('Loading cache')
    with open(hash_path, 'rb') as file:
        datasets = pickle.load(file)

Loading cache


In [5]:
#### Set Log ####
args['current_time'] = datetime.now().strftime('%Y%m%d')
args['LOG_NAME'] = get_log_name(args)

#### Update configs ####
if args.downsampling != 0: args['sampling_rate'] = args.downsampling

seed_everything(args.SEED)


Seed set to 42


LOG >>> Log name: 
	20231130_task_BCICompet2a_batch_512_lr_0.002_Baseline


42

In [6]:
for subject_id in datasets.keys(): 
    print(f"Subject {subject_id} has {len(datasets[subject_id])} trials")

Subject 0 has 288 trials
Subject 1 has 288 trials
Subject 2 has 288 trials
Subject 3 has 288 trials
Subject 4 has 288 trials
Subject 5 has 288 trials
Subject 6 has 288 trials
Subject 7 has 288 trials
Subject 8 has 288 trials


In [7]:
train_size = 240
val_size = 48


train_datasets = {}
val_datasets = {}
for subject_id in datasets.keys():
    train_datasets[subject_id] = torch.utils.data.Subset(datasets[subject_id], range(train_size))
    val_datasets[subject_id] = torch.utils.data.Subset(datasets[subject_id], range(train_size, train_size+val_size))


train_dataset_all = torch.utils.data.ConcatDataset(list(train_datasets.values()))
val_dataset_all = torch.utils.data.ConcatDataset(list(val_datasets.values()))
len(train_dataset_all), len(val_dataset_all)

(2160, 432)

In [8]:
train_dataloader_all = DataLoader(train_dataset_all, batch_size=args['batch_size'], shuffle=True, num_workers=0, persistent_workers=False)
val_dataloader_all = DataLoader(val_dataset_all, batch_size=args['batch_size'], shuffle=False, num_workers=0, persistent_workers=False)

In [9]:
model = ATTNConditioned(args, n_subjects=9, embed_dim=16,  n_classes=args['num_classes'] )
lit_model = LitModel(args, model)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv2d(input, weight, bias, self.stride,
/home/devuser/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [10]:
# import numpy as np
# eeg_input = torch.from_numpy(np.array([train_dataset_all[0]['data']]))
# subject_id =  torch.from_numpy(np.array([train_dataset_all[0]['subject_id']]))
# print(type(eeg_input), type(subject_id))
# print(eeg_input.shape, subject_id.shape)
# print(eeg_input.dtype, subject_id.dtype)

# model(eeg_input, subject_id)

In [11]:
logger = TensorBoardLogger(args.LOG_PATH, 
                                    name='all_attn_cond')

In [12]:
callbacks = get_callbacks(monitor='val_loss', args=args)

In [13]:
trainer = Trainer(
            max_epochs=args['EPOCHS'],
            callbacks=callbacks,
            default_root_dir=args.CKPT_PATH,
            logger=logger,
            enable_progress_bar=False
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(lit_model,
            train_dataloaders=train_dataloader_all,
            val_dataloaders=val_dataloader_all)
        
torch.cuda.empty_cache()

/home/devuser/.local/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /work/Nemes/m-shallowconvnet/checkpoints/20231130_task_BCICompet2a_batch_512_lr_0.002_Baseline exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/devuser/.local/lib/python3.8/site-packages/pytorch_lightning/core/optimizer.py:256: Found unsupported keys in the lr scheduler dict: {'internal'}. HINT: remove them from the output of `configure_optimizers`.

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ATTNConditioned  | 39.3 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
39.3 K    Trainable params
0         Non-trainable params
39.3 K    Total params
0.157     Total estimated model params size (MB)


  0%|          | 0/500 [00:00<?, ?it/s]

/home/devuser/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/devuser/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/devuser/.local/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=500` reached.
